<a href="https://colab.research.google.com/github/ghinanurazizah/IncomeifyApp-CH2-PS161/blob/ML/TensorFlow_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Import library
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Capstone Project/data.csv', sep='|')
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data Pre-processing**

In [5]:
# Hapus currency USD
df = df[df['salary_currency'] != 'USD']
# Drop kolom yang ga dipake
df.drop(['id', 'salary_currency', 'career_level', 'job_function', 'job_benefits', 'company_process_time', 'company_size', 'company_industry', 'job_description'], axis=1, inplace=True)
# Drop baris null
df = df.dropna()
# Hapus duplicate value
df.drop_duplicates()
df.head()

<ipython-input-5-1f1ae5c036fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['id', 'salary_currency', 'career_level', 'job_function', 'job_benefits', 'company_process_time', 'company_size', 'company_industry', 'job_description'], axis=1, inplace=True)


,job_title,location,experience_level,education_level,employment_type,salary
9,Tax Supervisor,Banten,3 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,8000000.0
18,Accounting Staff,Jakarta Pusat,1 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,4750000.0
22,Senior Staff Purchasing,Surabaya,2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,5250000.0
26,MARKETING COMMUNICATION & CHANNEL MANAGER,Jakarta Pusat,2 tahun,Tidak terspesifikasi,Penuh Waktu,15000000.0
28,Mechanic,Papua,1 tahun,Tidak terspesifikasi,Penuh Waktu,5040000.0


In [6]:
# Menghilangkan tahun pada experience_level
pattern = re.compile('[a-zA-Z]')
experience = [re.sub(pattern, '', s) for s in df['experience_level']]
# Mengganti nilai
df['experience_level'] = experience
# Mengganti tipe data
df['experience_level'] = df['experience_level'].astype(int)
df.head()

,job_title,location,experience_level,education_level,employment_type,salary
9,Tax Supervisor,Banten,3,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,8000000.0
18,Accounting Staff,Jakarta Pusat,1,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,4750000.0
22,Senior Staff Purchasing,Surabaya,2,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,5250000.0
26,MARKETING COMMUNICATION & CHANNEL MANAGER,Jakarta Pusat,2,Tidak terspesifikasi,Penuh Waktu,15000000.0
28,Mechanic,Papua,1,Tidak terspesifikasi,Penuh Waktu,5040000.0


In [7]:
# Hapus employment_type yang punya dua opsi
df = df[(df['employment_type'] != 'Penuh Waktu, Kontrak') & (df['employment_type'] != 'Penuh Waktu, Paruh Waktu') & (df['employment_type'] != 'Kontrak, Temporer')]
# Hapus education_level yang tidak terspesifikasi
df = df[df['education_level'] != 'Tidak terspesifikasi']

df.head()

,job_title,location,experience_level,education_level,employment_type,salary
9,Tax Supervisor,Banten,3,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,8000000.0
18,Accounting Staff,Jakarta Pusat,1,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,4750000.0
22,Senior Staff Purchasing,Surabaya,2,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,5250000.0
31,PPIC,Tangerang,5,Sarjana (S1),Penuh Waktu,6600000.0
34,C# Advanced/Senior Developer,Jakarta Timur,2,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,9500000.0


# **Removing outlier**

In [8]:
# Quantiles of experience level
Q1 = df['experience_level'].quantile(0.25)
Q3 = df['experience_level'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['experience_level'] > lower_bound) & (df['experience_level'] < upper_bound)]
df.head()

,job_title,location,experience_level,education_level,employment_type,salary
9,Tax Supervisor,Banten,3,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,8000000.0
18,Accounting Staff,Jakarta Pusat,1,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,4750000.0
22,Senior Staff Purchasing,Surabaya,2,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,5250000.0
31,PPIC,Tangerang,5,Sarjana (S1),Penuh Waktu,6600000.0
34,C# Advanced/Senior Developer,Jakarta Timur,2,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,9500000.0


# **Feature engineering**

**One-hot encoding**

In [9]:
# education_level yang harus dihapus
edu = ['Sertifikat Professional,', 'SMU/SMK/STM,', 'Diploma Pascasarjana,', 'Gelar Professional,', ', SMU/SMK/STM', 'Diploma', 'Sarjana', 'Magister', 'Doktor']
for i in edu:
    df['education_level'] = df['education_level'].str.replace(i, '')

# Hapus spasi ekstra
df['education_level'] = df['education_level'].apply(lambda x: ' '.join(x.split()))
# Hapus karakter ()
df['education_level'] = df['education_level'].str.replace(r'[()]', '')

# one-hot encoding on education_level
edu_encoded = df['education_level'].str.get_dummies(', ')
df = df.drop(columns=['education_level'])
# Merge dataframe
df_merged = pd.concat([df, edu_encoded], axis=1, ignore_index=False)
df_merged

<ipython-input-9-1efbc52dae94>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['education_level'] = df['education_level'].str.replace(r'[()]', '')


,job_title,location,experience_level,employment_type,salary,D3,D4,S1,S2,S3,SMA
9,Tax Supervisor,Banten,3,Penuh Waktu,8000000.0,1,1,1,0,0,0
18,Accounting Staff,Jakarta Pusat,1,Penuh Waktu,4750000.0,1,1,1,0,0,1
22,Senior Staff Purchasing,Surabaya,2,Penuh Waktu,5250000.0,1,1,1,0,0,0
31,PPIC,Tangerang,5,Penuh Waktu,6600000.0,0,0,1,0,0,0
34,C# Advanced/Senior Developer,Jakarta Timur,2,Penuh Waktu,9500000.0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
34741,Staff Accounting,Bekasi,2,Kontrak,4500000.0,0,0,1,0,0,0
34742,Frontend Developer,Jakarta Pusat,2,Penuh Waktu,11500000.0,0,0,1,0,0,0
34743,Account Manager,Jakarta Selatan,2,Penuh Waktu,6250000.0,0,0,1,0,0,0
34744,Sales,Palembang,1,Penuh Waktu,3387500.0,0,0,0,0,0,1


**Label encoding**

In [10]:
# Label encoding pada employment_type
le = LabelEncoder()
df_merged['employment_type'] = le.fit_transform(df_merged['employment_type'])
df_merged['location'] = le.fit_transform(df_merged['location'])
df_merged['job_title'] = le.fit_transform(df_merged['job_title'])

In [12]:
# Reset index
df = df_merged.reset_index(drop=True)
df

,job_title,location,experience_level,employment_type,salary,D3,D4,S1,S2,S3,SMA
0,4165,13,3,2,8000000.0,1,1,1,0,0,0
1,197,47,1,2,4750000.0,1,1,1,0,0,1
2,3711,148,2,2,5250000.0,1,1,1,0,0,0
3,2606,151,5,2,6600000.0,0,0,1,0,0,0
4,685,50,2,2,9500000.0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
6854,3850,18,2,0,4500000.0,0,0,1,0,0,0
6855,1440,47,2,2,11500000.0,0,0,1,0,0,0
6856,122,49,2,2,6250000.0,0,0,1,0,0,0
6857,3271,107,1,2,3387500.0,0,0,0,0,0,1


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6859 entries, 0 to 6858
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   job_title         6859 non-null   int64  
 1   location          6859 non-null   int64  
 2   experience_level  6859 non-null   int64  
 3   employment_type   6859 non-null   int64  
 4   salary            6859 non-null   float64
 5   D3                6859 non-null   int64  
 6   D4                6859 non-null   int64  
 7   S1                6859 non-null   int64  
 8   S2                6859 non-null   int64  
 9   S3                6859 non-null   int64  
 10  SMA               6859 non-null   int64  
dtypes: float64(1), int64(10)
memory usage: 589.6 KB


# **Build Model**

In [ ]:
# Ambil sampel dari data
sampled_data = df.sample(n=1000, random_state=42)

In [15]:
# Normalisasi menggunakan MinMaxScaler
scaler = MinMaxScaler()
normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

X = normalized[['job_title', 'location', 'experience_level', 'employment_type', 'D3 ', 'D4 ', 'S1', 'S2', 'S3', 'SMA']]
y = normalized['salary']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (5487, 10)
y_train shape: (5487,)
X_test shape: (1372, 10)
y_test shape: (1372,)


**Train Model**

In [16]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])

model.compile(loss='mse', optimizer=tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9), metrics=['mae'])

model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=100)

Epoch 1/100
172/172 [==============================] - 1s 4ms/step - loss: 0.0142 - mae: 0.1023 - val_loss: 0.0143 - val_mae: 0.1009
Epoch 2/100
172/172 [==============================] - 0s 3ms/step - loss: 0.0138 - mae: 0.1008 - val_loss: 0.0140 - val_mae: 0.0994
Epoch 3/100
172/172 [==============================] - 1s 3ms/step - loss: 0.0135 - mae: 0.0994 - val_loss: 0.0137 - val_mae: 0.0980
Epoch 4/100
172/172 [==============================] - 0s 2ms/step - loss: 0.0132 - mae: 0.0980 - val_loss: 0.0134 - val_mae: 0.0966
Epoch 5/100
172/172 [==============================] - 0s 2ms/step - loss: 0.0129 - mae: 0.0967 - val_loss: 0.0131 - val_mae: 0.0953
Epoch 6/100
172/172 [==============================] - 0s 2ms/step - loss: 0.0126 - mae: 0.0953 - val_loss: 0.0128 - val_mae: 0.0939
Epoch 7/100
172/172 [==============================] - 0s 2ms/step - loss: 0.0124 - mae: 0.0940 - val_loss: 0.0125 - val_mae: 0.0926
Epoch 8/100
172/172 [==============================] - 1s 3ms/step - 